### **Homework: Introduction**

**Q1: Running ElasticSearch**

Run elastic search using `curl localhost:9200` and get the cluster information. What's the `version.build_hash` value?

*"42f05b9372a9a4a470db3b52817899b99a76ee73"*

In [9]:
!curl localhost:9200

{
  "name" : "81a93798eb64",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "J56ntk96QgScGDnAWs_9XA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


**Q2: Indexing the Data**

Load, prepare and index the FAQ data as per the course videos. What function do you use to add your data to ElasticSearch?

*index*

In [10]:
import requests
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"}
        }
    }
}

index_name = "course-questions"
es = Elasticsearch("http://localhost:9200")

# create and index the documents only once
try:
    response = es.indices.create(index=index_name, body=index_settings)

    for doc in tqdm(documents):
        es.index(index=index_name, document=doc)
except:
    pass

/home/codespace/.local/share/virtualenvs/0_RAG_Workshop-Y-8dc-rB/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 948/948 [00:32<00:00, 29.23it/s]


**Q3: Searching**

Execute the query: *How do I execute a command in a running docker container?* Use only the `question` and `text` fields, give `question` a boost of 4, and use `"type": "best_fields"`. What's the score for the top-ranking result?

*84.05*

In [11]:
user_question = "How do I execute a command in a running docker container?"

search_query = {
    "size": 1,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
        }
    }
}

response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Course: {doc['course']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text']}...")
    print(f"Score: {hit['_score']}\n")


Course: machine-learning-zoomcamp
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)...
Score: 84.050095



**Q4: Filtering**

Limit the results to the top 3 from the *machine learning zoomcamp* course. What's the 3rd question returned?

*How do I copy files from a different folder into docker container's working directory?*

In [12]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": user_question,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
                        # search only the data engineering course documents
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es.search(index=index_name, body=search_query)

for hit in response['hits']['hits']:
    doc = hit['_source']
    print(f"Course: {doc['course']}")
    print(f"Question: {doc['question']}")
    print(f"Answer: {doc['text'][:100]}...\n")

Course: machine-learning-zoomcamp
Question: How do I debug a docker container?
Answer: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a ba...

Course: machine-learning-zoomcamp
Question: How do I copy files from my local machine to docker container?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here...

Course: machine-learning-zoomcamp
Question: How do I copy files from a different folder into docker container’s working directory?
Answer: You can copy files from your local machine into a Docker container using the docker cp command. Here...



**Q5: Building a Prompt**

Use the records returned from ElasticSearch in the previous question as the template to build the context. Separate context entries by a double-linebreak (`\n\n`). Then use that context along with the query from question 3 to construct a prompt. What's the length of that prompt?

*1,462 characters*

In [13]:
context_template = """
Q: {question}
A: {text}
""".strip()

context_docs = [hit['_source'] for hit in response['hits']['hits']]
context_result = ""

for doc in context_docs:
    doc_str = context_template.format(**doc)
    context_result += ("\n\n" + doc_str)

context = context_result.strip()

prompt = f"""
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {user_question}

CONTEXT:
{context}
""".strip()

print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [14]:
print(len(prompt))

1462


**Q6: Tokens**

OpenAI charges are based on the total number of tokens in the prompt and the response. The OpenAI python package uses `tiktoken` for tokenisation. How many tokens does the prompt in the previous question have?

*322 tokens*

In [19]:
import tiktoken

# get the tokeniser for the gpt-4o model
enc = tiktoken.encoding_for_model("gpt-4o")
# encode the prompt
encoded_prompt = enc.encode(prompt)
print(encoded_prompt)
print("\n", len(encoded_prompt))


[63842, 261, 4165, 14029, 29186, 13, 30985, 290, 150339, 4122, 402, 290, 31810, 8099, 591, 290, 40251, 7862, 558, 8470, 1606, 290, 19719, 591, 290, 31810, 8099, 1261, 55959, 290, 150339, 364, 107036, 25, 3253, 621, 357, 15792, 261, 6348, 306, 261, 6788, 62275, 9282, 1715, 10637, 50738, 734, 48, 25, 3253, 621, 357, 15199, 261, 62275, 9282, 3901, 32, 25, 41281, 290, 9282, 3621, 306, 25383, 6766, 326, 151187, 290, 7251, 4859, 11, 813, 484, 480, 13217, 261, 38615, 6348, 558, 68923, 2461, 533, 278, 2230, 7962, 4859, 38615, 464, 3365, 523, 3335, 290, 9282, 382, 4279, 6788, 11, 15792, 261, 6348, 306, 290, 4857, 9282, 734, 68923, 10942, 350, 6555, 290, 9282, 26240, 446, 68923, 25398, 533, 278, 464, 6896, 26240, 29, 38615, 198, 6103, 277, 10732, 391, 79771, 1029, 48, 25, 3253, 621, 357, 5150, 6291, 591, 922, 2698, 7342, 316, 62275, 9282, 3901, 32, 25, 1608, 665, 5150, 6291, 591, 634, 2698, 7342, 1511, 261, 91238, 9282, 2360, 290, 62275, 27776, 6348, 13, 44257, 1495, 316, 621, 480, 734, 1385, 51

*Bonus: Generating the Answer*

Send the prompt to Cohere. What's the response?

In [16]:
from cohere.client import Client

co_client = Client()

response = co_client.chat(
    message=prompt,
    model="command-r",
    preamble=""
    )
print(response.text)

To execute a command in a running docker container, first find the container ID by running the command docker ps. Then, run the command docker exec -it <container-id> bash to execute a bash command. Alternatively, you can also launch the container in interactive mode to execute commands.
